In [2]:
!pip3 install -q --upgrade pip
!pip3 install -q pandas numpy matplotlib seaborn openpyxl climateserv requests netCDF4 xarray pyproj geopandas Fiona
!pip3 install -q GDAL==3.8.2

In [3]:
from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd
from tqdm import tqdm

In [4]:
# Load your data
ndvi_df = pd.read_csv('../../data/processed_data/appears/ndvi/ndvi_data.csv')
monitoring_df = pd.read_csv('../../data/processed_data/igrac/igrac_gambia.csv')

In [5]:
ndvi_df

,Time,Longitude,Latitude,_250m_16_days_NDVI,_250m_16_days_EVI
0,2014-12-19 00:00:00,-17.078125,13.873958,NaN,NaN
1,2015-01-01 00:00:00,-17.078125,13.873958,NaN,NaN
2,2015-01-17 00:00:00,-17.078125,13.873958,NaN,NaN
3,2015-02-02 00:00:00,-17.078125,13.873958,NaN,NaN
4,2015-02-18 00:00:00,-17.078125,13.873958,NaN,NaN
...,...,...,...,...,...
112588590,2022-10-16 00:00:00,-13.769792,13.078125,0.5130,0.2398
112588591,2022-11-01 00:00:00,-13.769792,13.078125,0.5775,0.2711
112588592,2022-11-17 00:00:00,-13.769792,13.078125,0.5775,0.2711
112588593,2022-12-03 00:00:00,-13.769792,13.078125,0.5792,0.2673


In [6]:
monitoring_df

,ID,Value,Date,Day,Month,Year,Latitude,Longitude,Ground surface elevation,LIS_Soil_Moisture_Combined
0,NWP00001,14.26,2016-04-14,14,4,2016,13.420804,-16.716666,19.47,0.172124
1,NWP00001,13.82,2015-12-26,26,12,2015,13.420804,-16.716666,19.47,0.210837
2,NWP00001,13.83,2015-12-26,26,12,2015,13.420804,-16.716666,19.47,0.210837
3,NWP00001,13.82,2015-12-25,25,12,2015,13.420804,-16.716666,19.47,0.211614
4,NWP00001,13.82,2015-12-25,25,12,2015,13.420804,-16.716666,19.47,0.211614
...,...,...,...,...,...,...,...,...,...,...
25401,NWP62002,28.90,2015-12-29,29,12,2015,13.425272,-14.206073,36.35,0.236943
25402,NWP62002,29.19,2015-09-29,29,9,2015,13.425272,-14.206073,36.35,0.316936
25403,NWP62002,29.59,2015-07-29,29,7,2015,13.425272,-14.206073,36.35,0.239098
25404,NWP62002,29.44,2015-05-29,29,5,2015,13.425272,-14.206073,36.35,0.197747


In [8]:
ndvi_df.dtypes

Time                   object
Longitude             float64
Latitude              float64
_250m_16_days_NDVI    float64
_250m_16_days_EVI     float64
dtype: object

In [9]:
monitoring_df.dtypes

ID                             object
Value                         float64
Date                           object
Day                             int64
Month                           int64
Year                            int64
Latitude                      float64
Longitude                     float64
Ground surface elevation      float64
LIS_Soil_Moisture_Combined    float64
dtype: object

In [10]:
import pandas as pd
from sklearn.neighbors import BallTree
import numpy as np

# Assume ndvi_df and monitoring_df are your dataframes

# Convert Time and Date columns to datetime
ndvi_df['Time'] = pd.to_datetime(ndvi_df['Time'])
monitoring_df['Date'] = pd.to_datetime(monitoring_df['Date'])

# Create spatial keys by rounding latitudes and longitudes
ndvi_df['spatial_key'] = ndvi_df.apply(lambda row: f"{round(row['Latitude'], 2)}_{round(row['Longitude'], 2)}", axis=1)
monitoring_df['spatial_key'] = monitoring_df.apply(lambda row: f"{round(row['Latitude'], 2)}_{round(row['Longitude'], 2)}", axis=1)

In [4]:
import pandas as pd

# Sample data in your format (replace with your actual data)
ndvi_data = {
    'Time': ['2014-12-19', '2015-01-01', '2015-01-17', '2015-02-02', '2015-02-18'],
    'Longitude': [-17.078125, -17.078125, -17.078125, -17.078125, -17.078125],
    'Latitude': [13.873958, 13.873958, 13.873958, 13.873958, 13.873958],
    '_250m_16_days_NDVI': [1, 2, 3, 4, 5],
    '_250m_16_days_EVI': [1, 2, 3, 4, 5]
}

monitoring_data = {
    'ID': ['NWP00001', 'NWP00001', 'NWP00001', 'NWP00001', 'NWP00001'],
    'Value': [14.26, 13.82, 13.83, 13.82, 13.82],
    'Date': ['2016-04-14', '2015-12-26', '2015-12-26', '2015-12-25', '2015-12-25'],
    'Latitude': [13.420804, 13.420804, 13.420804, 13.420804, 13.420804],
    'Longitude': [-16.716666, -16.716666, -16.716666, -16.716666, -16.716666]
}

ndvi_df = pd.DataFrame(ndvi_data)
monitoring_df = pd.DataFrame(monitoring_data)

# Convert Time and Date columns to datetime
ndvi_df['Time'] = pd.to_datetime(ndvi_df['Time'])
monitoring_df['Date'] = pd.to_datetime(monitoring_df['Date'])

# Create spatial keys
ndvi_df['spatial_key'] = ndvi_df.apply(lambda row: f"{round(row['Latitude'], 2)}_{round(row['Longitude'], 2)}", axis=1)
monitoring_df['spatial_key'] = monitoring_df.apply(lambda row: f"{round(row['Latitude'], 2)}_{round(row['Longitude'], 2)}", axis=1)

# Select the first 10 rows of monitoring_df for testing
monitoring_df_subset = monitoring_df.head(10)

# Function to find nearest date
def find_nearest_date(row):
    same_spatial_key = ndvi_df[ndvi_df['spatial_key'] == row['spatial_key']]
    if same_spatial_key.empty:
        return np.nan, np.nan  # Return NaN if no spatial match is found

    closest_idx = np.abs(same_spatial_key['Time'] - row['Date']).idxmin()
    return same_spatial_key.loc[closest_idx, '_250m_16_days_NDVI'], same_spatial_key.loc[closest_idx, '_250m_16_days_EVI']

# Apply the function to each row in the subset of monitoring_df
monitoring_df_subset[['NDVI', 'EVI']] = monitoring_df_subset.apply(find_nearest_date, axis=1, result_type='expand')

# Display the modified subset
print(monitoring_df_subset)


         ID  Value       Date   Latitude  Longitude   spatial_key  NDVI  EVI
0  NWP00001  14.26 2016-04-14  13.420804 -16.716666  13.42_-16.72   NaN  NaN
1  NWP00001  13.82 2015-12-26  13.420804 -16.716666  13.42_-16.72   NaN  NaN
2  NWP00001  13.83 2015-12-26  13.420804 -16.716666  13.42_-16.72   NaN  NaN
3  NWP00001  13.82 2015-12-25  13.420804 -16.716666  13.42_-16.72   NaN  NaN
4  NWP00001  13.82 2015-12-25  13.420804 -16.716666  13.42_-16.72   NaN  NaN


/var/folders/mb/tcvp4z490sz6dt5nlf1qwh580000gn/T/ipykernel_28685/715770734.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monitoring_df_subset[['NDVI', 'EVI']] = monitoring_df_subset.apply(find_nearest_date, axis=1, result_type='expand')
/var/folders/mb/tcvp4z490sz6dt5nlf1qwh580000gn/T/ipykernel_28685/715770734.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monitoring_df_subset[['NDVI', 'EVI']] = monitoring_df_subset.apply(find_nearest_date, axis=1, result_type='expand')
